## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

In [1]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=4
physical_network="exogeni"

#### Create a Lease

In [2]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
lease = get_lease(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-05-06T19:51:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "8930cdb4-a9b6-4fc0-ac40-b402bfc908fb",
      "resource_id": "2f8561d4-a9de-4ce9-9fec-8851a75746b9",
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "created_at": "2020-05-05 19:51:52",
      "updated_at": "2020-05-05 19:51:53",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "26912164-3382-4f7e-9d47-2681e56e6642",
      "resource_type": "virtual:floatingip",
      "resources_changed": false
    },
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "8930cdb4-a9b6-4fc0-ac40-b402bfc908fb",
      "resource_id": "a28ca845-a778-4b9a-beb3-ff64d33417f9",
      "max": 4,
      "created_at": "2020-05-05 19:51:43",
      "min": 4,
      "updated_at": "2020-05-05 19:51:50",
      "missing_

#### Get the Reservations

In [3]:
#Get the lease by name
lease = get_lease(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

compute_reservation_id: 7a7a8f7d-e8af-44ed-b09f-459ece770cbf
network_reservation_id: 9ddfc970-2249-4e89-8179-f08872462b43
floatingip_reservation_id: 26912164-3382-4f7e-9d47-2681e56e6642


#### Configute the Network

In [5]:
network = get_network_by_name(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

Network ID: 2e814389-d160-49cf-9f94-9ab9ba70b784


In [6]:
subnet = add_subnet(subnet_name, network_name)
router = create_router(router_name, network_name)
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': '2e814389-d160-49cf-9f94-9ab9ba70b784',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '736482b2-eb5e-40fa-aba5-4d7afcf14a50',
 'subnet_ids': ['736482b2-eb5e-40fa-aba5-4d7afcf14a50'],
 'port_id': '5b1872f3-609b-4148-9647-b20a84fdd277',
 'id': 'b73d8319-8ff1-4a2e-add5-b5fa0982aded'}

#### Start the Server

In [7]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)

TypeError: Object of type Server is not JSON serializable

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [2]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

[{'router_id': None, 'status': 'DOWN', 'description': '', 'tags': ['blazar', 'reservation:69b1e298-bdde-4059-a46a-9c4e234fa30e'], 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2', 'created_at': '2020-05-05T16:57:12Z', 'updated_at': '2020-05-05T16:57:13Z', 'floating_network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'port_details': None, 'fixed_ip_address': None, 'floating_ip_address': '192.5.87.218', 'revision_number': 1, 'project_id': 'd9faac3973a847f1b718fa765fe312e2', 'port_id': None, 'id': '24db1b1e-d530-4e03-b872-bc4071a4e927'}, {'router_id': None, 'status': 'DOWN', 'description': '', 'tags': ['blazar', 'reservation:26912164-3382-4f7e-9d47-2681e56e6642'], 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2', 'created_at': '2020-05-05T19:52:06Z', 'updated_at': '2020-05-05T19:52:07Z', 'floating_network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'port_details': None, 'fixed_ip_address': None, 'floating_ip_address': '192.5.87.204', 'revision_number': 1, 'project_id': 'd9faac3973a847f1b7

#### Delete Server

In [3]:
delete_server_by_name(server_name)

()

#### De-configure Network

In [4]:
try:
    result = detach_router_by_name(router_name=router_name, subnet_name=subnet_name)
except Exception as e:
    print("detach_router_by_name error:" + str(e))
    pass

try:
    result = delete_router_by_name(router_name)
except Exception as e:
    print("delete_router_by_name error: " + str(e))
    pass

try:
    result = delete_subnet_by_name(subnet_name)
except Exception as e:
    print("delete_subnet_by_name error: " + str(e))
    pass

try:
    result = delete_network_by_name(network_name)
except Exception as e:
    print("delete_network_by_name error: " + str(e))
    pass

#### Release Lease

In [5]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id 8930cdb4-a9b6-4fc0-ac40-b402bfc908fb
